Basic analysis to iHMP_IBDMDB_2019. The overlap (21 metabolites) with OSA. 

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.multitest import fdrcorrection
import matplotlib.pyplot as plt
import seaborn as sns
from preprocessors import MetabolitePreprocessor


Metabolite mapped to HMDB - v. Taxa mapped using GTDB - v.
This research data is HMP2 cohort - longitudinal samples from IBD patient and control.
The metabolom is untargeted. Microbiome is shotgun. 
There are 382 samples for 105 patient. For control we have 104 samples for 26 subjects.
We will take only control and all samples (ignoring longitudinal). 


In [5]:
human_db_path = '/home/noa/lab_code/microbiome-metabolome-curated-data/data/processed_data'
dataset = 'iHMP_IBDMDB_2019'

metadata = pd.read_csv(f'{human_db_path}/{dataset}/metadata.tsv' ,sep='\t')
metabolite = pd.read_csv(f'{human_db_path}/{dataset}/mtb.tsv' ,sep='\t')
metabolite_to_hmdb = pd.read_csv(f'{human_db_path}/{dataset}/mtb.map.tsv' ,sep='\t')
taxa = pd.read_csv(f'{human_db_path}/{dataset}/genera.tsv' ,sep='\t')

In [6]:
metabolite_to_hmdb = metabolite_to_hmdb.set_index('Compound')['HMDB']

Metadata overview: Study.Group:
CD - Crohn's disease
UC - ulcreative colitis. 
nonIBD - control 

In [7]:
# Filter to control samples only:
control_samples_ids = metadata[metadata['Study.Group'] == 'nonIBD'].Sample.unique()
taxa = taxa[taxa.Sample.isin(control_samples_ids)]
metabolite = metabolite[metabolite.Sample.isin(control_samples_ids)]


Taxa is in genus level in relative abundance. 
No need to normalize, but do need to filter rare taxa. 

In [8]:
# Filter rare taxa (taxa have non-zero values in >10% of the samples)")
# Filter rare taxa according to the following criteria: we want that the at least 10% of the samples will have relative-abundance larger than 0.001.

taxa = taxa.set_index('Sample')
verbose=False
percentage=10
abundance_threshold=0.001
min_number_of_samples = int((taxa.shape[0] / 100) * percentage)
print(
    f"The number of genus/features that have relative abundance values larger then {abundance_threshold} on > {percentage} % samples are: "
    f"{(((taxa > abundance_threshold).sum(axis=0)) >= min_number_of_samples).sum()}"
    f" out of {taxa.shape[1]} genus/features before-filtering.")

non_rare_columns = taxa.columns[
    ((taxa > abundance_threshold).sum(axis=0)) >= min_number_of_samples]
taxa = taxa[non_rare_columns]

In [9]:
# taxa_mean_abundance_across_samples = taxa.mean()

In [10]:
# taxa_mean_abundance_across_samples.plot.hist(bins=100)

In [11]:
# pd.cut(taxa_mean_abundance_across_samples, bins=20, retbins=True)
# taxa_mean_abundance_across_samples.describe()

In [12]:
# print(f"{np.count_nonzero(taxa_mean_abundance_across_samples < 0.01)} out of {taxa_mean_abundance_across_samples.shape[0]} samples have mean-relative abundance below 0.01%. \n"
#       f" AKA {round((np.count_nonzero(taxa_mean_abundance_across_samples < 0.01)) / taxa_mean_abundance_across_samples.shape[0] * 100, 2)} % consider rare. Filtering using this threshold will leave us with: {( taxa_mean_abundance_across_samples.shape[0] - (np.count_nonzero(taxa_mean_abundance_across_samples < 0.01)))} genus.")

In [13]:
# So maybe selecting the mean in such diverse dataset is too much? maybe we should take other quantile? 
# Maybe we can say that we want that in 10% of the samples it will have relative-abundance above x? 
# Or that in the median? 
# taxa_q10_abundance_across_samples = taxa.quantile(q=0.9)


In [14]:
# taxa_q10_abundance_across_samples.plot.hist(bins=100)


In [15]:
# taxa_q10_abundance_across_samples.describe()

In [16]:
# taxa_q10_abundance_across_samples.sort_values()


In [17]:
# print(f"{np.count_nonzero(taxa_q10_abundance_across_samples < 0.01)} out of {taxa_q10_abundance_across_samples.shape[0]} samples have quantile 90% relative abundance below 0.01%. \n"
#       f" AKA {round((np.count_nonzero(taxa_q10_abundance_across_samples < 0.01)) / taxa_q10_abundance_across_samples.shape[0] * 100, 2)} % consider rare. Filtering using this threshold will leave us with: {( taxa_q10_abundance_across_samples.shape[0] - (np.count_nonzero(taxa_q10_abundance_across_samples < 0.01)))} genus.")

In [18]:
# print(f"{np.count_nonzero(taxa_q10_abundance_across_samples < 0.001)} out of {taxa_q10_abundance_across_samples.shape[0]} samples have quantile 90% relative abundance below 0.001%. \n"
#       f" AKA {round((np.count_nonzero(taxa_q10_abundance_across_samples < 0.001)) / taxa_q10_abundance_across_samples.shape[0] * 100, 2)} % consider rare. Filtering using this threshold will leave us with: {( taxa_q10_abundance_across_samples.shape[0] - (np.count_nonzero(taxa_q10_abundance_across_samples < 0.001)))} genus.")

Filter rare taxa according to the following criteria: we want that the at least 10% of the samples will have relative-abundance larger than 0.001.

In [19]:
metabolite = metabolite.set_index('Sample')

In [20]:
# metabolite.columns.isin(metabolite_to_hmdb.index).all()
metabolite = metabolite[metabolite.columns.intersection(metabolite_to_hmdb.dropna().index)].rename(columns=metabolite_to_hmdb.dropna())

Map compound to HMDB (article metabolite name in the original data). As it can be many to one, we will sum according to HMDB.

In [21]:
metabolite = metabolite.groupby(by=metabolite.columns, axis=1).sum()

In [22]:
metabolite.shape

Filter rare metabolite

In [23]:
percentage = 85
min_number_of_samples = int((metabolite.shape[0] / 100) * percentage)
min_number_of_samples

In [24]:
metabolite.shape

In [25]:
metabolite = metabolite.fillna(0)

In [26]:
(metabolite.round(decimals=8) == 0).sum(axis=0)

In [27]:
non_rare_columns = metabolite.columns[((metabolite.shape[0] - (
            metabolite.round(decimals=8) == 0).sum(axis=0)) >= min_number_of_samples)]
print(f"There are {len(non_rare_columns)} metabolite with sufficient number of samples "
      f"(>{percentage}%) out of {metabolite.shape[1]} metabolites.")
metabolite= metabolite[non_rare_columns]

Pre-processing metabolite data:

In [28]:
# hmdb_metadata = pd.read_xml('/home/noa/lab_code/H2Mtranslation/data/feces_metabolites.xml')
# map_hmdb_id_to_name = hmdb_metadata.set_index('accession', drop=True)['name']
# map_hmdb_id_to_name.to_pickle('/home/noa/lab_code/H2Mtranslation/data/map_hmdb_id_to_name.pkl')
# hmdb_metadata.set_index('accession', drop=True)[['name', 'description']].to_pickle('/home/noa/lab_code/H2Mtranslation/data/hmdb_name_and_description.pkl')


In [29]:
map_hmdb_id_to_name = pd.read_pickle('//map_hmdb_id_to_name.pkl')

In [30]:
hmdb_metadata = pd.read_pickle('//hmdb_name_and_description.pkl')


In [31]:
min_value_per_metabolite = metabolite.replace(to_replace=0, value=np.nan).min(axis=0) / 2
metabolite.replace(to_replace=np.nan, value=0, inplace=True)
metabolite.replace(to_replace=0, value=min_value_per_metabolite, inplace=True)
metabolite = metabolite.apply(lambda x: np.log(x + 1))

In [32]:
metabolite = metabolite.rename(columns=map_hmdb_id_to_name)

In [33]:
metabolite.head()

Correlation test:
For each metabolite independently, and for each taxa independently, (aka for each (metabolite, genus) tuple) calculate correlation across samples. 

In [34]:
# Do we have 'Unknown' in our taxa? no. They drop unknown - is it problematic? As in mice we do have unknown... But it's really different %. 

In [35]:
metabolome_bacteria_corr_coefficient = pd.DataFrame(index=taxa.columns, columns=metabolite.columns)
metabolome_bacteria_p_values = pd.DataFrame(index=taxa.columns, columns=metabolite.columns)

for metabolome in metabolite.columns:
    for bacteria in taxa.columns:
        coefficient, p  = stats.pearsonr(metabolite[metabolome], taxa[bacteria])
        metabolome_bacteria_corr_coefficient.loc[bacteria, metabolome] = coefficient
        metabolome_bacteria_p_values.loc[bacteria, metabolome] = p
        

In [36]:
# With multi-comparison correction (FDR)
# what to do with nans?
p_values = metabolome_bacteria_p_values.stack()
_, p_values_corrected = fdrcorrection(p_values)
metabolome_bacteria_p_values_fdr_correction = pd.Series(p_values_corrected, index=p_values.index).unstack()

In [37]:
metabolome_bacteria_p_values_fdr_correction

In [38]:
metabolome_bacteria_corr_coefficient = metabolome_bacteria_corr_coefficient.T.stack()
metabolome_bacteria_p_values_fdr_correction = metabolome_bacteria_p_values_fdr_correction.T.stack()

In [39]:
metabolome_bacteria_significant_corr = metabolome_bacteria_corr_coefficient[metabolome_bacteria_p_values_fdr_correction < 0.05]

In [40]:
print(f"There are {metabolome_bacteria_significant_corr.shape[0]} significant couples out of {metabolome_bacteria_corr_coefficient.shape[0]} "
      f" ({taxa.shape[1]} genus, {metabolite.shape[1]} metabolome) "
      f"after FDR correction (aka {round(100 * metabolome_bacteria_significant_corr.shape[0]/ metabolome_bacteria_corr_coefficient.shape[0] ,2)} %.)")

In [41]:
metabolome_bacteria_significant_corr.sort_values(ascending=False)


In [42]:
metabolome_bacteria_significant_corr.to_pickle(f'{dataset}/metabolome_bacteria_significant_corr_draft1.pkl')
metabolome_bacteria_corr_coefficient.to_pickle(f'{dataset}/metabolome_bacteria_corr_coefficient_draft1.pkl')
metabolome_bacteria_p_values_fdr_correction.to_pickle(f'{dataset}/metabolome_bacteria_p_values_fdr_correction_draft1.pkl')

In [43]:
taxa_rename_to_genus = pd.Series(metabolome_bacteria_significant_corr.index.get_level_values(1), index=metabolome_bacteria_significant_corr.index.get_level_values(1)).str.extract('.*g__(.*)').squeeze()

In [44]:
taxa_rename_to_genus.head()

In [45]:
metabolome_bacteria_significant_corr = metabolome_bacteria_significant_corr.rename(taxa_rename_to_genus.to_dict(), axis=0, level=1)

In [46]:
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

top_metabolite = metabolome_bacteria_significant_corr.sort_values(ascending=False).index[0][0]
idx = pd.IndexSlice
metabolome_bacteria_significant_corr.loc[idx[top_metabolite, :]].sort_values(ascending=False).plot.bar(figsize=(4,3), title=top_metabolite)

In [47]:
bottom_metabolite = metabolome_bacteria_significant_corr.sort_values(ascending=False).index[-1][0]
metabolome_bacteria_significant_corr.loc[idx[bottom_metabolite, :]].sort_values(ascending=False).plot.bar(figsize=(12,6), title=bottom_metabolite)

In [48]:
# How many bacteria are significantly correlated with metabolites?
print(f"We have {len(metabolome_bacteria_significant_corr.index.get_level_values(1).unique())} microbes with significant correlation to metabolite out of "
      f"{taxa.shape[1]} metabolites ({round(len(metabolome_bacteria_significant_corr.index.get_level_values(1).unique()) / taxa.shape[1] * 100)}%)")

In [49]:
# Search for significant bacteria:
# Number of metabolites each bacteria significantly effects on
metabolome_bacteria_significant_corr.groupby(axis=0, level=1).apply(lambda x: x.shape[0]).sort_values(ascending=False)

In [50]:
metabolome_bacteria_significant_corr.groupby(axis=0, level=1).apply(lambda x: x.shape[0]).describe()

We can see that the #metabolites that correlate with the bacteria is highley divers (std 17) where most bacteria effect up to 4 metabolites, while some correlates with up to 86. 

In [51]:
# How many metabolism highly-derived from bacteria over-all the metabolites in the DB?
print(f"We have {len(metabolome_bacteria_significant_corr.index.get_level_values(0).unique())} metabolite with significant correlation to microbiome out of "
      f"{metabolite.shape[1]} metabolites ({round(len(metabolome_bacteria_significant_corr.index.get_level_values(0).unique()) / metabolite.shape[1] * 100)}%)")

Number of bacteria each metabolite have significant correlation with:

In [52]:
metabolome_bacteria_significant_corr.groupby(axis=0, level=0).apply(lambda x: x.shape[0]).sort_values(ascending=False).describe()


we mostly see 1/2 bacteria is significantly associate to the metabolite, 
were sometimes we see multiple bacteria associate with the metabolite.

In [53]:
top_bacteria = metabolome_bacteria_significant_corr.groupby(axis=0, level=1).apply(lambda x: x.shape[0]).idxmax()
metabolome_bacteria_significant_corr.loc[idx[:, top_bacteria]].sort_values(ascending=False).plot.bar(figsize=(12,6), title=f'{top_bacteria} significant correlation to metabolites')


In [54]:
metabolites_feces_from_microbial_origin_hmdb = pd.read_csv('//metabolites_feces_from_microbial_origin_hmdb')
metabolites_feces_from_microbial_origin_hmdb = metabolites_feces_from_microbial_origin_hmdb.set_index('HMDB_ID')['NAME']

Compare the overlap between the metabolites that we have find as significantly-correlated with microbiome and HMDB list of 117 feces metabolite that origin from microbial.

In [55]:
metabolome_bacteria_significant_corr.index.get_level_values(0).unique().intersection(metabolites_feces_from_microbial_origin_hmdb.unique())

In [56]:
metabolome_bacteria_significant_corr.index.get_level_values(0).unique().intersection(metabolites_feces_from_microbial_origin_hmdb.unique()).shape[0]

How many of the feces metabolite that origin from microbial (HMDB list) are in our datasets (with enough data?)

In [57]:
metabolite.columns.intersection(metabolites_feces_from_microbial_origin_hmdb.unique()).shape[0]

In [58]:
23/28

So we identified 23/28 (82%) as associate to microbiome, (overall to 75% of the 375 metabolite we measure have correlation. 
How is it relative to randomly sampling 28 metabolite out of the 375 with significant tag?) 

How the relationship look like on the 5 that we didn't identify as microbiome driven? 
How this 23 look like with respect to the rest? 
How many unknown/known metabolite do I have in this list?  


In [59]:
# sample 28 out of 375 only with the tag significant/not. And calculate the groups % of positive. 
is_metabolite_correlates = pd.Series(metabolite.columns, index=metabolite.columns).apply(lambda x: x in metabolome_bacteria_significant_corr.index.get_level_values(0).unique())
is_metabolite_correlates

In [60]:
exp_ = [round(is_metabolite_correlates.sample(n=28).mean() * 100, 2) for _ in range(1000)]

In [61]:
pd.Series(exp_).describe()

What do we know on the 23 metabolites (fecal from microbial origin according to HMDB) #TODO

In [62]:
import re
taxa_genus = taxa.rename(columns=lambda x: re.match('.*g__(.*)', x).group(1))

In [63]:
metabolome_bacteria_p_values_fdr_correction = metabolome_bacteria_p_values_fdr_correction.rename(taxa_rename_to_genus.to_dict(), axis=0, level=1)

sanity checks - for some significant correlation between microbiome-metabolome plot the relation across the samples 
see how much I believe this correlation analysis & how much it's effected by outliers. 

In [64]:
metabolite_1, microbiome_1 = metabolome_bacteria_significant_corr.sort_values(ascending=False).index[0]
pd.concat([metabolite[metabolite_1], taxa_genus[microbiome_1]], axis=1).plot.scatter(x=microbiome_1, y=metabolite_1, figsize=(6,6), 
                                                                                     title = f'corr coef={round(metabolome_bacteria_significant_corr[(metabolite_1, microbiome_1)],2)}, p-value={"{:.2e}".format(metabolome_bacteria_p_values_fdr_correction[(metabolite_1, microbiome_1)])}')

In [65]:
metabolite[metabolite_1][metabolite[metabolite_1] > 18].index[0]

In [66]:
coefficient, p  = stats.pearsonr(metabolite[metabolite_1].drop('MSM6J2M3'), taxa_genus[microbiome_1].drop('MSM6J2M3'))
pd.concat([metabolite[metabolite_1].drop('MSM6J2M3'), taxa_genus[microbiome_1].drop('MSM6J2M3')], axis=1).plot.scatter(x=microbiome_1, y=metabolite_1, figsize=(6,6), 
                                                                                     title = f'corr coef={round(coefficient,2)}, p-value={"{:.2e}".format(p)}')

In [67]:
metabolite_1, microbiome_1 = metabolome_bacteria_significant_corr.sort_values(ascending=False).index[2]
pd.concat([metabolite[metabolite_1], taxa_genus[microbiome_1]], axis=1).plot.scatter(x=microbiome_1, y=metabolite_1, figsize=(6,6), 
                                                                                     title = f'corr coef={round(metabolome_bacteria_significant_corr[(metabolite_1, microbiome_1)],2)}, p-value={"{:.2e}".format(metabolome_bacteria_p_values_fdr_correction[(metabolite_1, microbiome_1)])}')

In [68]:
outliers = taxa_genus[microbiome_1][taxa_genus[microbiome_1] > 0.006].index.to_list()


In [69]:
outliers

In [70]:
# outliers = metabolite[metabolite_1][metabolite[metabolite_1] < 12].index.to_list()
coefficient, p  = stats.pearsonr(metabolite[metabolite_1].drop(outliers), taxa_genus[microbiome_1].drop(outliers))
pd.concat([metabolite[metabolite_1].drop(outliers), taxa_genus[microbiome_1].drop(outliers)], axis=1).plot.scatter(x=microbiome_1, y=metabolite_1, figsize=(6,6), 
                                                                                     title = f'corr coef={round(coefficient,2)}, p-value={"{:.2e}".format(p)}')

In [71]:
# metabolome_bacteria_corr_coefficient 
# metabolome_bacteria_p_values_fdr_correction

In [72]:
#  For unknown (without a name, and known) taxa, plot a scatter plot of the correlations to see if it’s due to some outliers or do I believe it. (metabolite vs. taxa across the 90 samples + in the title mention the correlation coefficient) 

# Plot also metabolite vs. taxa across the 90 samples of metabolite that are known to be microbial origin (the 5 I didn't got as significant)? what does it even mean? read a bit more?

# The most interest question I think is why I didn't get the other 5 microbial origin metabolite as metabolite with correlation to microbiota?
metabolites_group = metabolite.columns.intersection(metabolites_feces_from_microbial_origin_hmdb.unique()).difference(metabolome_bacteria_significant_corr.index.get_level_values(0))
print(metabolites_group)

In [73]:
hmdb_metadata.query('name in @metabolites_group').set_index('name')['description'].iloc[0]

In [74]:
metabolite['2-Hydroxyglutarate'].plot.hist(figsize=(6,6))

In [75]:
# before grouping by and other pre-processing:
# 
# metabolite_org = pd.read_csv(f'{human_db_path}/{dataset}/mtb.tsv' ,sep='\t')
# metabolite_org = metabolite_org[metabolite_org.Sample.isin(control_samples_ids)]
# metabolite_org = metabolite_org.set_index('Sample')
# metabolite_org = metabolite_org[metabolite_org.columns.intersection(metabolite_to_hmdb.dropna().index)].rename(columns=metabolite_to_hmdb.dropna())
# metabolite_org = metabolite_org.rename(columns=map_hmdb_id_to_name)


In [76]:
# metabolite_org[metabolites_group]

Top 5 bacteria (that correlates with a lot of metabolites):  

In [77]:
metabolome_bacteria_significant_corr.groupby(axis=0, level=1).apply(lambda x: x.shape[0]).sort_values(ascending=False).iloc[:5].index.to_list()

Internet: 
Alistipes - Gram-negative genus in the phylum Bacteridota. Highly associated with our health. 
 https://en.wikipedia.org/wiki/Alistipes
 https://www.sciencedirect.com/topics/immunology-and-microbiology/alistipes
 https://www.frontiersin.org/journals/immunology/articles/10.3389/fimmu.2020.00906/full
 Indeed involve in a lot of pathways:
https://www.genome.jp/kegg-bin/show_organism?menu_type=pathway_maps&category=Alistipes


Our data: 
* per bacteria, show the relative-abundance across the 90 samples. (distribution)

In [78]:
taxa_genus['Alistipes'].plot.hist(figsize=(4,6), title='Alistipes')

In [79]:
taxa_genus['Alistipes'].describe()

In [80]:
# TODO: How much the result changes/ how does they look like when I consider sperman/ kendall correlation? 

In [81]:
sns.set_theme(rc={'figure.figsize':(20,14), 'xtick.labelsize':5, 'ytick.labelsize':5})
sns.heatmap((metabolome_bacteria_p_values_fdr_correction.unstack() < 0.05).astype(int)).set(title='Heatmap significant or not correlation of Metabolite & Bacteria')

In [82]:
sns.heatmap(metabolome_bacteria_significant_corr.unstack().astype(float)).set(title='Heatmap significant correlation of Metabolite & Bacteria')
